In [18]:
using Pkg
Pkg.activate(@__DIR__);
pkg"instantiate"
pkg"precompile"

using RigidBodyDynamics
using MeshCatMechanisms

  Activating environment at `/mnt/064AC6424AC62E6D/git_workspace/FlyHopper/scripts/Julia/Project.toml`


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-7856550021708782407\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-14810451528458425668\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-7856550021708782407\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [34]:
curdir = pwd()
const URDFPATH = joinpath(curdir, "../../src/res/flyhopper_mockup/urdf/flyhopper_mockup.urdf")
bot = parse_urdf(Float64, URDFPATH)
remove_fixed_tree_joints!(bot);

In [5]:
g = -9.81 # gravitational acceleration in z-direction
state = MechanismState(bot)
q = configuration(state)
v = velocity(state)

2-element SegmentedVector{JointID, Float64, Base.OneTo{JointID}, Vector{Float64}}:
 0.0
 0.0

In [6]:
center_of_mass(state)

Point3D in "world": [-0.12990371873117074, -1.706979234022984e-6, -0.15000031810601624]

In [15]:
q0, q1 = joints(bot)
function simple_control!(torques::AbstractVector, t, state::MechanismState)
    torques[velocity_range(state, q0)] .= -1 .* velocity(state, q0)
    torques[velocity_range(state, q1)] .= 10 * sin(t)
end;

In [16]:
final_time = 10.
ts, qs, vs = simulate(state, final_time, simple_control!; Δt = 1e-3);

In [43]:
zero_velocity!(state)
# set_configuration!(state, q0, 0.7)
# set_configuration!(state, q1, -0.8);
mvis = MechanismVisualizer(bot, URDFVisuals(URDFPATH));
MeshCatMechanisms.animate(mvis, ts, qs; realtimerate = 1.);

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://localhost:8711
└ @ MeshCat /home/ben/.julia/packages/MeshCat/X2AUA/src/visualizer.jl:73
┌ Warning: Could not find the mesh file: src/res/flyhopper_mockup/meshes/base_link.obj. Also tried changing the extension to .obj.
└ @ MechanismGeometries.URDF /home/ben/.julia/packages/MechanismGeometries/DSouH/src/urdf.jl:72
┌ Warning: Could not find the mesh file: package://../res/flyhopper_mockup/meshes/Link 1.STL. Tried substituting the following folders for the 'package://' prefix: [""].
│ Also tried changing the extension to .obj.
└ @ MechanismGeometries.URDF /home/ben/.julia/packages/MechanismGeometries/DSouH/src/urdf.jl:58
┌ Warning: Could not find the mesh file: package://../res/flyhopper_mockup/meshes/Link 2.STL. Tried substituting the following folders for the 'package://' prefix: [""].
│ Also tried changing the extension to .obj.
└ @ MechanismGeometries.URDF /home/ben/.jul